In [43]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
import pickle

DATA_FILE3 = 'data/pos_adjectives_list.pkl'
DATA_FILE4 = 'data/neu_adjectives_list.pkl'
DATA_FILE5 = 'data/neg_adjectives_list.pkl'

DATA_FILE = 'data/data.pkl'
with open(DATA_FILE, 'rb') as f:
    Cs = pickle.load(f)
    
DATA_FILE2 = 'data/labels.pkl'  
with open(DATA_FILE2, 'rb') as f:
    Ys = pickle.load(f)
    
with open(DATA_FILE3, 'rb') as f:
    pos_adjectives_list = pickle.load(f)
    
with open(DATA_FILE4, 'rb') as f:
    neu_adjectives_list = pickle.load(f)

with open(DATA_FILE5, 'rb') as f:
    neg_adjectives_list = pickle.load(f)

In [16]:
print(neu_adjectives_list)

['total', 'not', 'strawberry', 'filthy', 'less', 'unsatisfactory', 'regular', 'chipotle', 'first', 'so', 'average', 'be', 'full', 'enough', 'available', 'stodgy', 'extra', 'new', 'different', 'slow', 'mostly', 'only', 'fast', 'unable', 'large', 'slowest', 'next', 'early', 'really', 'nt', 'last', 'entire', 'aerobic', 'young', 'the', 'its', 'mediocre', 'plain', 'loud', 'administrative', 'small', 'other', 'late', 'own', 'bearded', 'ice', 'second', 'lasagnacold', 'white', '10', 'open', 'sour', 'a', 'sashmi', 'raw', 'expensive', 'previous', 'african', '2nd', 'due', 'close', 'third', 'busy', 'female', 'real', 'asian', 'old', 'same', 'also', 'despicable', 'final', 'dysfunctional', 'thai', 'unattentive', 'purple', 'near', 'biggest', 'male', 'inattentive', 'right', 'impossible', 'greasy', 'catty', 'typical', 'more', 'soggy', 'square', 'rip', 'absolutely', 'lackluster', 'hot', 'very', 'short', 'usual', 'quick', 'accurate', 'wet', 'cold', 'was', 'doable', 'iced', 'at', 'dry', 'decent', 'many', 'l

In [17]:
import collections

collections.Counter(Ys[0])

Counter({1: 463, 2: 470, 3: 468, 4: 473, 5: 463})

In [18]:
print(f"Train Size: {len(Cs[0])}")
print(f"Dev Size:   {len(Cs[1])}")
print(f"Test Size:  {len(Cs[2])}")
print(f"Train Size: {len(Ys[0])}")
print(f"Dev Size:   {len(Ys[1])}")
print(f"Test Size:  {len(Ys[2])}")

Train Size: 2337
Dev Size:   887
Test Size:  887
Train Size: 2337
Dev Size:   887
Test Size:  895


In [ ]:
candidate = Cs[0][901]
print(f"Sentence:\n{candidate.text}")
print(f"Candidate:\nX: {candidate[0]}\nY: {candidate[1]}\nZ: {candidate[2]}\nP: {candidate[3]}")

In [45]:
from babble.utils import ExplanationIO

FILE = "data/my_explanations.tsv"
exp_io = ExplanationIO()
explanations = exp_io.read(FILE)

Read 5199 explanations from data/my_explanations.tsv


In [7]:
print(explanations[0].candidate)

RelationMention(doc_id=1: entities=("total"(0:5), "bill"(6:10), "service"(29:36))


In [46]:
from babble import Babbler
babbler = Babbler(Cs, Ys)

Grammar construction complete.


In [47]:
babbler.add_aliases({'positive': pos_adjectives_list})
babbler.add_aliases({'negative': neg_adjectives_list})

Grammar construction complete.
Grammar construction complete.


In [48]:
babbler.apply(explanations, split=0)

- | #                                                 | 2 Elapsed Time: 0:00:00

Building list of target candidate ids...
Collected 3830 unique target candidate ids from 5199 explanations.
Gathering desired candidates...
Could not find 3830 target candidates with the following mention_ids (first 5):
213974578855038195
4461466229265486721
375898482870210384
-7521498117606266766
-2934910685688459680
Found 0/3830 desired candidates
Linking explanations to candidates...
Linked 0/5199 explanations


| | #                                              | 5198 Elapsed Time: 0:05:07


5199 explanation(s) out of 5199 were parseable.
10398 parse(s) generated from 5199 explanation(s).
3398 parse(s) remain (7000 parse(s) removed by DuplicateSemanticsFilter).
Note: 3398 LFs did not have candidates and therefore could not be filtered.
3398 parse(s) remain (0 parse(s) removed by ConsistencyFilter).
Applying labeling functions to investigate labeling signature.
[========================================] 100%

Added 3398 parse(s) from 1699 explanations to set. (Total # parses = 3398)

Applying labeling functions to split 1
[========================================] 100%

Added 35760 labels to split 1: L.nnz = 35760, L.shape = (894, 3398).
Applying labeling functions to split 2
[========================================] 100%

Added 35760 labels to split 2: L.nnz = 35760, L.shape = (894, 3398).


In [49]:
Ls = []
for split in [0,1,2]:
    L = babbler.get_label_matrix(split)
    Ls.append(L)

Retrieved label matrix for split 0: L.nnz = 374280, L.shape = (9357, 3398)
Retrieved label matrix for split 1: L.nnz = 35760, L.shape = (894, 3398)
Retrieved label matrix for split 2: L.nnz = 35760, L.shape = (894, 3398)


In [ ]:
parses = babbler.get_parses(translate=False)
print(len(parses))
for parse in parses:
    babbler.view_parse(parse)
    print(parse.function(candidate))

In [26]:
from metal import LabelModel

label_aggregator = LabelModel(5)
label_aggregator.train(Ls[0], n_epochs=100, lr=0.01)
label_aggregator.score(Ls[1], Ys[1])

Computing O...
Estimating \mu...


/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.


[E:0]	Train Loss: 38568.828


/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype

[E:10]	Train Loss: 1118.895


...

/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype

[E:90]	Train Loss: 0.696


/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype

[E:99]	Train Loss: 0.177
Finished Training
Accuracy: 0.061


0.060879368658399095

In [27]:
Y_p = label_aggregator.predict(Ls[0])

In [28]:
len_wrong_train = 0
for right, wrong in zip(Ys[0], Y_p):
    if right != wrong:
        len_wrong_train += 1
print(len_wrong_train)

1867


In [50]:
def most_frequent(line):
    filtered_list = list(filter(lambda a: a != 0, line))
    if len(filtered_list) > 0:
        occurence_count = Counter(filtered_list)
        return occurence_count.most_common(1)[0][0]
    else:
        return 0


def percentage(part, whole):
    return 100 * float(part)/float(whole)

In [51]:
from collections import Counter

L_train = Ls[0].toarray()
L_test = Ls[2].toarray()

predicted_training_labels = []
predicted_test_labels = []

for line in L_train:
    predicted_training_labels.append(most_frequent(line))

for line in L_test:
    predicted_test_labels.append(most_frequent(line))

perturbations = []

len_wrong_train = 0
for right, wrong in zip(Ys[0], predicted_training_labels):
    if int(right) != wrong:
        len_wrong_train += 1
    perturbations.append(int(right) - wrong)

perturbations = [abs(perturbation) for perturbation in perturbations]

len_wrong_test = 0
for right, wrong in zip(Ys[2], predicted_test_labels):
    if int(right) != wrong:
        len_wrong_test += 1

training_accuracy = percentage(len(Ys[0]) -len_wrong_train,len(Ys[0]))

test_accuracy = percentage(len(Ys[2]) - len_wrong_test, len(Ys[2]))

print("Number of wrong in training set: " + str(len_wrong_train))
print("Number of wrong in test set: " + str(len_wrong_test))
print("Training Accuracy: " + str(training_accuracy))
print("Test Accuracy " + str(test_accuracy))

Number of wrong in training set: 7451
Number of wrong in test set: 687
Training Accuracy: 20.369776637811263
Test Accuracy 24.004424778761063


In [33]:
print(most_frequent(L_train[0]))

4


In [ ]:
wrong_indexes = []
i = 0
for right,wrong in zip(Ys[0],Y_p):
    if int(right) != wrong:
        wrong_indexes.append(i)
    i = i + 1

In [ ]:
print(len(wrong_indexes))

In [ ]:
with open("Ls.pkl", 'wb') as f:
    pickle.dump(Ls, f)
    
with open("Y_p.pkl", 'wb') as f:
    pickle.dump(Y_p, f)
    
with open("data/wrong_indexes.pkl", 'wb') as f:
    pickle.dump(wrong_indexes, f)